In [2]:
import carla
import numpy as np
import cv2
import time
IM_WIDTH = 640
IM_HEIGHT = 480
# Connect to CARLA server
client = carla.Client('localhost', 2000)
client.set_timeout(5.0)
# Get the world and blueprint library
world = client.get_world()
blueprint_library = world.get_blueprint_library()

In [1]:
# Spawn a vehicle
vehicle_bp = blueprint_library.filter('model3')[0]
spawn_point = carla.Transform(carla.Location(x=0, y=137, z=0.1), carla.Rotation(pitch=0.0, yaw=180.0, roll=0.0))
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# Attach a camera sensor to the vehicle
camera_bp = blueprint_library.find('sensor.camera.rgb')
camera_bp.set_attribute("image_size_x", f"{IM_WIDTH}")
camera_bp.set_attribute("image_size_y", f"{IM_HEIGHT}")
camera_transform = carla.Transform(carla.Location(x=2.0, z=1.5), carla.Rotation(pitch=-15.0))
camera = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

seg_camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
seg_camera_bp.set_attribute("image_size_x", f"{IM_WIDTH}")
seg_camera_bp.set_attribute("image_size_y", f"{IM_HEIGHT}")
seg_camera_transform = carla.Transform(carla.Location(x=2.0, z=1.5), carla.Rotation(pitch=-15.0))

# Spawn the camera and attach to the vehicle
seg_camera = world.spawn_actor(seg_camera_bp, seg_camera_transform, attach_to=vehicle)

def process_image(image):
    try:
        i = np.array(image.raw_data)
        #print(i.shape)
        i2 = i.reshape((IM_HEIGHT, IM_WIDTH, 4))
        i3 = i2[:, :, :3]
        gray = cv2.cvtColor(i3, cv2.COLOR_BGR2GRAY)
        return gray
    except Exception as e:
        print("Error in process_image:", str(e))
        return None

def detect_lanes(gray):
    try:
        display_image = gray.copy()
        # mask_white = cv2.inRange(gray, 175, 255)
        blurred = cv2.GaussianBlur(gray, (9, 9), 0)
        # Edge Detection
        edges = cv2.Canny(blurred, 70, 70)
        height, width = edges.shape
        roi_vertices = [
            (0, height),
            (0, height * 0.3), # left (x, y)
            (width, height * 0.3), # right (x, y)
            (width, height)
        ]

        mask = np.zeros_like(edges)
        cv2.fillPoly(mask, [np.array(roi_vertices, dtype=np.int32)], 255)
        roi = cv2.bitwise_and(edges, mask)

        lines = cv2.HoughLinesP(roi, 1, np.pi/180, threshold=100, minLineLength=50, maxLineGap=100)
        black_image = np.zeros((height, width, 3), np.uint8)
        if lines is not None:
            for line in lines:
                for x1, y1, x2, y2 in line:
                    cv2.line(black_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
                    cv2.line(display_image, (x1, y1), (x2, y2), (255, 255, 255), 2)

        # Original Image Display
        # cv2.imshow('Edges', edges)
        # cv2.imshow('ROI', roi)
        # cv2.imshow("Mask", mask)
        # cv2.imshow("mask white", mask_white)
        # cv2.imshow("black", black_image)
        cv2.imshow("Original", display_image)
        cv2.waitKey(1)

        # Compute lane center based on lines
        if lines is not None and len(lines) > 1:
            left_lane, right_lane = lines[0][0], lines[1][0]
            lane_center = ((left_lane[0] + right_lane[0]) / 2, (left_lane[1] + right_lane[1]) / 2)
            # print(f"left_lane: {left_lane}, right_lane: {right_lane}, lane_center: {lane_center}")
        else:
            lane_center = (width / 2, height)
        
        return lane_center
    except Exception as e:
        print("Error in detect_lanes:", str(e))
    return (400, 300)

def process_semantic_image(image):
    # Convert the raw data to a numpy array
    semantic_array = np.array(image.raw_data).reshape((IM_HEIGHT, IM_WIDTH, 4))
    
    # Extract only the R channel (which contains the class labels)
    labels = semantic_array[:, :, 2]

    # Define a colormap to represent different classes (e.g., red for vehicles, green for pedestrians, etc.)
    # Adjust this colormap as per your needs.
    colormap = {
        # 0: [0, 0, 0],        # None
        1: [70, 70, 70],    # road
        # 2: [190, 153, 153], # sidewalks
        # 3: [250, 170, 160], # building
        # 4: [220, 20, 60],   # not sure
        # 5: [153, 153, 153], # not sure
        # 6: [157, 234, 50],  # light poles
        # 7: [128, 64, 128],  # traffic lights
        # 8: [244, 35, 232],  # TrafficSign
        # 10: [0, 0, 142],    # not sure
        # 11: [244, 35, 232],    # sky
        # 14: [244, 35, 232],    # cars
        # 20: [244, 35, 232],    # Dynamic
        # 22: [244, 35, 232],    # Dynamic
        # Add other classes as needed
    }

    # Convert labels to RGB based on the colormap
    semantic_rgb = np.zeros((IM_HEIGHT, IM_WIDTH, 3))
    for key, value in colormap.items():
        semantic_rgb[np.where(labels == key)] = value

    return semantic_rgb.astype(np.uint8)
# Define a function for steering control
def control_vehicle(lane_center):
    try:
        # _, width, _ = world.get_settings().screen_resolution
        car_center = IM_WIDTH / 2.0

        deviation = lane_center[0] - car_center
        kp = 0.0007
        steering = -kp * deviation
        # print(steering)
        vehicle.apply_control(carla.VehicleControl(throttle=0.5, steer=steering))
    except Exception as e:
        print("Error in control_vehicle:", str(e))

def callback(image):
    semantic_processed = process_semantic_image(image)
    # cv2.imshow("Semantic Processed", semantic_processed)

    # processed_image = process_image(image)
    processed_image = cv2.cvtColor(semantic_processed, cv2.COLOR_BGR2GRAY)
    # cv2.imshow("Grayscale", processed_image)

    lane_center = detect_lanes(processed_image)
    # print("Callback called. Lane center:", lane_center)
    control_vehicle(lane_center)

def semantic_callback(image):
    # processed_image = process_image(image)
    processed_image = process_semantic_image(image)
    cv2.imshow("Semantic Segmentation", processed_image)
    cv2.waitKey(1)

# Listen to camera images, detect lanes, and control the vehicle
# camera.listen(callback)
seg_camera.listen(callback)


In [16]:
# Start from a known location on the track
start_location = carla.Location(x=0, y=137, z=0.1)  # Change to your desired starting location
waypoint_api = map.get_waypoint(start_location)

waypoint_list = []
next_waypoint = waypoint_api
distance = 20.0  # distance between consecutive waypoints
num_waypoints = 120  # Number of waypoints to generate, adjust as needed

for _ in range(num_waypoints):
    # Append to the list
    waypoint_list.append(next_waypoint)

    # Print x and y coordinates
    print("X:", round(next_waypoint.transform.location.x), " Y:", round(next_waypoint.transform.location.y))
    # world.debug.draw_point(next_waypoint.transform.location, size=0.1, color=carla.Color(255,0,0), life_time=5)
    world.debug.draw_line(vehicle.get_transform().location, next_waypoint.transform.location, thickness=0.05, color=carla.Color(0,0,255), life_time=5)

    # Get the next waypoint
    next_waypoint = next_waypoint.next(distance)[0]

X: 0  Y: 137
X: 20  Y: 137
X: 40  Y: 138
X: 60  Y: 138
X: 80  Y: 134
X: 96  Y: 121
X: 105  Y: 103
X: 106  Y: 82
X: 106  Y: 62
X: 106  Y: 42
X: 106  Y: 22
X: 106  Y: 2
X: 106  Y: -18
X: 104  Y: -38
X: 92  Y: -55
X: 73  Y: -64
X: 53  Y: -64
X: 33  Y: -64
X: 13  Y: -64
X: -7  Y: -65
X: -27  Y: -65
X: -46  Y: -57
X: -49  Y: -37
X: -49  Y: -17
X: -48  Y: 4
X: -39  Y: 22
X: -19  Y: 25
X: 1  Y: 25
X: 21  Y: 25
X: 41  Y: 25
X: 61  Y: 25
X: 81  Y: 25
X: 102  Y: 21
X: 110  Y: 2
X: 110  Y: -18
X: 107  Y: -39
X: 94  Y: -58
X: 74  Y: -67
X: 53  Y: -68
X: 33  Y: -68
X: 13  Y: -68
X: -7  Y: -68
X: -27  Y: -68
X: -47  Y: -68
X: -67  Y: -69
X: -89  Y: -64
X: -106  Y: -49
X: -113  Y: -28
X: -114  Y: -8
X: -114  Y: 12
X: -114  Y: 32
X: -114  Y: 52
X: -115  Y: 72
X: -113  Y: 94
X: -105  Y: 114
X: -91  Y: 131
X: -71  Y: 140
X: -50  Y: 140
X: -30  Y: 141
X: -10  Y: 141
X: 10  Y: 141
X: 30  Y: 141
X: 50  Y: 141
X: 71  Y: 141
X: 92  Y: 132
X: 105  Y: 114
X: 109  Y: 92
X: 109  Y: 72
X: 110  Y: 52
X: 110  Y: 32

In [11]:
# For each waypoint you want to visualize:
world.debug.draw_point(carla.Location(x=0, y=137, z=0.1), size=0.1, color=carla.Color(255,0,0), life_time=10)

In [15]:
world.debug.clear()

AttributeError: 'DebugHelper' object has no attribute 'clear'